The ReturnFlows component will eventually handle both the withdrawals and return flows between counties and gauges.  It provides the link between `gauges`, which are nodes in the Water Network, and `canals`, which are the bipartite graph edges that connect `gauges` and `counties`.

Currently the ReturnFlows component does not handle return flows.  It does, however, handle withdrawals.

In [1]:
using Mimi

@defcomp ReturnFlows begin
    canals = Index()

    # How much to send from each gauge to each county
    withdrawals = Parameter(index=[canals, time])

    # For now, exact copy of withdrawals; later, the amount actually provided for each withdrawal?
    copy_withdrawals = Variable(index=[canals, time])
    # Water removed from gauge
    removed = Variable(index=[gauges, time])
end

ReturnFlows

The ReturnFlows component takes as its input the withdrawals specified on a per-canal basis.

In [2]:
function timestep(c::ReturnFlows, tt::Int)
    v = c.Variables
    p = c.Parameters
    d = c.Dimensions

    for gg in 1:numgauges
        v.removed[gg, tt] = 0.
    end

    for pp in 1:nrow(draws)
        v.copy_withdrawals[pp, tt] = p.withdrawals[pp, tt]
        if p.withdrawals[pp, tt] > 0
            gaugeid = draws[pp, :gaugeid]
            gg = findfirst(collect(keys(wateridverts)) .== gaugeid)
            if (gg == 0)
                println("Missing $gaugeid")
            else
                v.removed[gg, tt] += p.withdrawals[pp, tt]
            end
        end
    end
end

timestep (generic function with 3 methods)

The `timestep` sums up all water removed from each gauge.

It would be nice for the ReturnFlows component to take `withdrawals` as "requested withdrawals", and then to produce a variable describing "statisfied withdrawals", dependent on the availability of river flow at the gauges.  However, since ReturnFlows is run before the WaterNetwork, the full propogation of withdrawals and additions is unknown.

The simulation resolves this by applying additions and withdrawals simultaneously all throughout the water network.  However, this logic is more complicated and difficult to debug.

Future tasks:
* Check that in simulation mode, WaterNetwork `added` is incremental, since this specifies that `removed` is incremental.
* Extract the `gauge x canal` portion of the WaterNetwork's `grad_waternetwork_outflows_withdrawals` to this component.
* Handle actual return flows.